In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [12]:
def prep(path='KE30_Steel_C04.csv'):
    data=pd.read_csv(path)
    print('Inital Data Frame= '+str(data.shape[0])+' rows')



    df=data[['Invoice Date','Customer Code','Material Code','Actual Volume','Customer Name','Material Description']]
    df.dropna(inplace=True)
    df['Total']=df.groupby(['Invoice Date','Customer Code','Material Code'])['Actual Volume'].transform('sum')
    
    df.drop(columns='Actual Volume',inplace=True)



    cc=np.unique(df['Customer Code'])
    df_new= pd.DataFrame(columns=df.columns)
    c=0
    for i in cc:
        df_test=df[df['Customer Code']==i]

        arr=df_test['Total']
        cv=np.std(arr)/np.mean(arr)
        print('CV = '+str(cv)+ ' for '+str(i))
        if cv>1 :
            print('Deleting '+str(i))
            df_new=df_new.append(df[df['Customer Code']==i] , ignore_index=True)
            df=df[df['Customer Code']!=i]   
            print('-'*20)
        print('xx'*49)


    df.to_csv('Table1.csv',index=False)
    df_new.to_csv('Table2.csv',index=False)


In [2]:
dic_a={}
dic_p={}

In [3]:
def testo(path='Table2.csv'):
    df12=pd.read_csv(path)
    data=df12[['Invoice Date','Customer Code','Material Code','Total']]

    
    for i in data['Customer Code']:
        
        df=data[data['Customer Code']==i]
        
        date=pd.to_datetime('20190220', format='%Y%m%d', errors='ignore')
        df['Invoice Date']=pd.to_datetime(df['Invoice Date'])
        
        
        train=df[df['Invoice Date']<=date]
        test=df[df['Invoice Date']>date]
        train.drop(columns=['Material Code','Customer Code'],inplace=True)
        test.drop(columns=['Material Code','Customer Code'],inplace=True)
      
        train['Total_']=df.groupby(['Invoice Date'])['Total'].transform('sum')
        train = train.drop_duplicates(subset=['Invoice Date'])
        test['Total_']=df.groupby(['Invoice Date'])['Total'].transform('sum')
        test = test.drop_duplicates(subset=['Invoice Date'])
        
        train.index=train['Invoice Date']
        train.drop(columns=['Invoice Date','Total'],inplace=True)
        test.index=test['Invoice Date']
        y_actual=test['Total']
        test.drop(columns=['Invoice Date','Total'],inplace=True)
        
        #print(train)
        
        #print(test)
        
        def create_dataset(dataset, look_back=1):
            dataX, dataY = [], []
            for j in range(len(dataset)-look_back-1):
                a = dataset[j:(j+look_back), 0]
                dataX.append(a)
                dataY.append(dataset[j + look_back, 0])
            return np.array(dataX), np.array(dataY)


        look_back = 1
        X_train, Y_train = create_dataset(np.array(train), look_back)
        X_test, Y_test = create_dataset(np.array(test), look_back)

        # reshape input to be [samples, time steps, features]
        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
        model = Sequential()
        model.add(LSTM(4, input_shape=(1, look_back)))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=1)
        y_pred=model.predict(X_test)
        dic_p.update({i:y_pred})
        dic_a.update({i:y_actual})
    
        

In [ ]:
testo()

Train on 32 samples
Epoch 1/100
32/32 [==============================] - 3s 89ms/sample - loss: 247676.4231
Epoch 2/100
32/32 [==============================] - 0s 3ms/sample - loss: 247629.9847
Epoch 3/100
32/32 [==============================] - 0s 2ms/sample - loss: 247583.6398
Epoch 4/100
32/32 [==============================] - 0s 2ms/sample - loss: 247536.9369
Epoch 5/100
32/32 [==============================] - 0s 3ms/sample - loss: 247490.3139
Epoch 6/100
32/32 [==============================] - 0s 3ms/sample - loss: 247443.3680
Epoch 7/100
32/32 [==============================] - 0s 3ms/sample - loss: 247397.6276
Epoch 8/100
32/32 [==============================] - 0s 3ms/sample - loss: 247351.1549
Epoch 9/100
32/32 [==============================] - 0s 3ms/sample - loss: 247303.7983
Epoch 10/100
32/32 [==============================] - 0s 3ms/sample - loss: 247258.4620
Epoch 11/100
32/32 [==============================] - 0s 3ms/sample - loss: 247210.9369
Epoch 12/100
32/32 [

Epoch 94/100
32/32 [==============================] - 0s 3ms/sample - loss: 243372.8912
Epoch 95/100
32/32 [==============================] - 0s 2ms/sample - loss: 243327.3378
Epoch 96/100
32/32 [==============================] - 0s 3ms/sample - loss: 243281.0016
Epoch 97/100
32/32 [==============================] - 0s 3ms/sample - loss: 243235.1403
Epoch 98/100
32/32 [==============================] - 0s 3ms/sample - loss: 243189.5565
Epoch 99/100
32/32 [==============================] - 0s 3ms/sample - loss: 243142.6562
Epoch 100/100
32/32 [==============================] - 0s 3ms/sample - loss: 243097.3857
Train on 32 samples
Epoch 1/100
32/32 [==============================] - 2s 77ms/sample - loss: 247469.8318
Epoch 2/100
32/32 [==============================] - 0s 2ms/sample - loss: 247423.8117
Epoch 3/100
32/32 [==============================] - 0s 2ms/sample - loss: 247376.4077
Epoch 4/100
32/32 [==============================] - 0s 3ms/sample - loss: 247330.9420
Epoch 5/100
32

Epoch 87/100
32/32 [==============================] - 0s 3ms/sample - loss: 243490.2794
Epoch 88/100
32/32 [==============================] - 0s 3ms/sample - loss: 243444.6654
Epoch 89/100
32/32 [==============================] - 0s 3ms/sample - loss: 243398.2573
Epoch 90/100
32/32 [==============================] - 0s 3ms/sample - loss: 243352.2865
Epoch 91/100
32/32 [==============================] - 0s 3ms/sample - loss: 243305.8629
Epoch 92/100
32/32 [==============================] - 0s 3ms/sample - loss: 243261.2108
Epoch 93/100
32/32 [==============================] - 0s 3ms/sample - loss: 243214.0162
Epoch 94/100
32/32 [==============================] - 0s 3ms/sample - loss: 243168.9439
Epoch 95/100
32/32 [==============================] - 0s 3ms/sample - loss: 243123.0197
Epoch 96/100
32/32 [==============================] - 0s 3ms/sample - loss: 243076.7322
Epoch 97/100
32/32 [==============================] - 0s 3ms/sample - loss: 243030.9121
Epoch 98/100
32/32 [============

Epoch 80/100
32/32 [==============================] - 0s 3ms/sample - loss: 244146.5652
Epoch 81/100
32/32 [==============================] - 0s 3ms/sample - loss: 244101.3143
Epoch 82/100
32/32 [==============================] - 0s 3ms/sample - loss: 244055.6413
Epoch 83/100
32/32 [==============================] - 0s 3ms/sample - loss: 244008.9640
Epoch 84/100
32/32 [==============================] - 0s 3ms/sample - loss: 243963.1334
Epoch 85/100
32/32 [==============================] - 0s 2ms/sample - loss: 243917.0480
Epoch 86/100
32/32 [==============================] - 0s 3ms/sample - loss: 243870.3430
Epoch 87/100
32/32 [==============================] - 0s 3ms/sample - loss: 243825.1638
Epoch 88/100
32/32 [==============================] - 0s 3ms/sample - loss: 243778.5663
Epoch 89/100
32/32 [==============================] - 0s 3ms/sample - loss: 243732.7641
Epoch 90/100
32/32 [==============================] - 0s 3ms/sample - loss: 243687.2590
Epoch 91/100
32/32 [============

Epoch 73/100
32/32 [==============================] - 0s 3ms/sample - loss: 244551.7848
Epoch 74/100
32/32 [==============================] - 0s 3ms/sample - loss: 244505.6890
Epoch 75/100
32/32 [==============================] - 0s 3ms/sample - loss: 244459.3581
Epoch 76/100
32/32 [==============================] - 0s 3ms/sample - loss: 244413.7682
Epoch 77/100
32/32 [==============================] - 0s 3ms/sample - loss: 244367.2975
Epoch 78/100
32/32 [==============================] - 0s 3ms/sample - loss: 244321.3684
Epoch 79/100
32/32 [==============================] - 0s 3ms/sample - loss: 244275.6793
Epoch 80/100
32/32 [==============================] - 0s 3ms/sample - loss: 244229.4018
Epoch 81/100
32/32 [==============================] - 0s 3ms/sample - loss: 244183.1904
Epoch 82/100
32/32 [==============================] - 0s 3ms/sample - loss: 244136.8583
Epoch 83/100
32/32 [==============================] - 0s 3ms/sample - loss: 244091.1605
Epoch 84/100
32/32 [============

...

32/32 [==============================] - 0s 3ms/sample - loss: 243442.4473
Epoch 92/100
32/32 [==============================] - 0s 3ms/sample - loss: 243395.0746
Epoch 93/100
32/32 [==============================] - 0s 3ms/sample - loss: 243348.8363
Epoch 94/100
32/32 [==============================] - 0s 3ms/sample - loss: 243302.8384
Epoch 95/100
32/32 [==============================] - 0s 3ms/sample - loss: 243257.0831
Epoch 96/100
32/32 [==============================] - 0s 3ms/sample - loss: 243210.6491
Epoch 97/100
32/32 [==============================] - 0s 3ms/sample - loss: 243164.0865
Epoch 98/100
32/32 [==============================] - 0s 3ms/sample - loss: 243119.0263
Epoch 99/100
32/32 [==============================] - 0s 3ms/sample - loss: 243072.2930
Epoch 100/100
32/32 [==============================] - 0s 3ms/sample - loss: 243025.7523
Train on 32 samples
Epoch 1/100
32/32 [==============================] - 4s 121ms/sample - loss: 247889.1365
Epoch 2/100
32/32 [====

32/32 [==============================] - 0s 3ms/sample - loss: 240758.7358
Epoch 85/100
32/32 [==============================] - 0s 3ms/sample - loss: 240673.6250
Epoch 86/100
32/32 [==============================] - 0s 3ms/sample - loss: 240588.3239
Epoch 87/100
32/32 [==============================] - 0s 3ms/sample - loss: 240504.3595
Epoch 88/100
32/32 [==============================] - 0s 3ms/sample - loss: 240420.7755
Epoch 89/100
32/32 [==============================] - 0s 3ms/sample - loss: 240333.0712
Epoch 90/100
32/32 [==============================] - 0s 3ms/sample - loss: 240251.0551
Epoch 91/100
32/32 [==============================] - 0s 3ms/sample - loss: 240165.1804
Epoch 92/100
32/32 [==============================] - 0s 3ms/sample - loss: 240080.3008
Epoch 93/100
32/32 [==============================] - 0s 3ms/sample - loss: 239995.4797
Epoch 94/100
32/32 [==============================] - 0s 3ms/sample - loss: 239911.5444
Epoch 95/100
32/32 [=========================

32/32 [==============================] - 0s 3ms/sample - loss: 243968.1191
Epoch 78/100
32/32 [==============================] - 0s 3ms/sample - loss: 243921.3193
Epoch 79/100
32/32 [==============================] - 0s 3ms/sample - loss: 243876.3347
Epoch 80/100
32/32 [==============================] - 0s 3ms/sample - loss: 243829.9552
Epoch 81/100
32/32 [==============================] - 0s 3ms/sample - loss: 243783.2539
Epoch 82/100
32/32 [==============================] - 0s 3ms/sample - loss: 243738.3039
Epoch 83/100
32/32 [==============================] - 0s 3ms/sample - loss: 243691.7726
Epoch 84/100
32/32 [==============================] - 0s 3ms/sample - loss: 243645.7596
Epoch 85/100
32/32 [==============================] - 0s 3ms/sample - loss: 243599.7797
Epoch 86/100
32/32 [==============================] - 0s 3ms/sample - loss: 243554.7047
Epoch 87/100
32/32 [==============================] - 0s 3ms/sample - loss: 243507.2737
Epoch 88/100
32/32 [=========================

32/32 [==============================] - 0s 3ms/sample - loss: 245117.1659
Epoch 71/100
32/32 [==============================] - 0s 3ms/sample - loss: 245071.1222
Epoch 72/100
32/32 [==============================] - 0s 3ms/sample - loss: 245023.9067
Epoch 73/100
32/32 [==============================] - 0s 3ms/sample - loss: 244979.1783
Epoch 74/100
32/32 [==============================] - 0s 3ms/sample - loss: 244931.8784
Epoch 75/100
32/32 [==============================] - 0s 3ms/sample - loss: 244886.5274
Epoch 76/100
32/32 [==============================] - 0s 3ms/sample - loss: 244840.1606
Epoch 77/100
32/32 [==============================] - 0s 3ms/sample - loss: 244793.7410
Epoch 78/100
32/32 [==============================] - 0s 3ms/sample - loss: 244747.4383
Epoch 79/100
32/32 [==============================] - 0s 3ms/sample - loss: 244701.9908
Epoch 80/100
32/32 [==============================] - 0s 3ms/sample - loss: 244654.8011
Epoch 81/100
32/32 [=========================

32/32 [==============================] - 0s 3ms/sample - loss: 244845.9693
Epoch 64/100
32/32 [==============================] - 0s 3ms/sample - loss: 244800.1470
Epoch 65/100
32/32 [==============================] - 0s 3ms/sample - loss: 244754.5648
Epoch 66/100
32/32 [==============================] - 0s 3ms/sample - loss: 244707.5448
Epoch 67/100
32/32 [==============================] - 0s 3ms/sample - loss: 244661.7762
Epoch 68/100
32/32 [==============================] - 0s 3ms/sample - loss: 244616.0793
Epoch 69/100
32/32 [==============================] - 0s 3ms/sample - loss: 244569.2202
Epoch 70/100
32/32 [==============================] - 0s 3ms/sample - loss: 244523.5763
Epoch 71/100
32/32 [==============================] - 0s 3ms/sample - loss: 244477.2413
Epoch 72/100
32/32 [==============================] - 0s 3ms/sample - loss: 244431.5417
Epoch 73/100
32/32 [==============================] - 0s 3ms/sample - loss: 244384.9635
Epoch 74/100
32/32 [=========================